# Formation South Green 2021  

##  Initiation à l’analyse de données Minion

### PART 2

Created by J.Orjuela (DIADE-IRD), F. Sabot (DIADE-IRD) and G. Sarah (AGAP-CIRAD)

Septembre 2021


## 1. Assemblies

More contiguous genome assemblies can be generated using long sequencing read but assembly is not a quite pace. Eukaryotic genomes assembly is a complex task (large genome sizes, high rates of repeated sequences, high heterozygosity levels and even polyploidy), and while prokaryotic genomes may appear less challenging, specific features such as circular DNA molecules, must be taken into consideration to achieve high quality assembly.

* For assembly, ONT recommend sequencing a human genome to a minimum depth of 30x of 25–35 kb reads.
However, sequencing to a depth of 60x is advisable to obtain the best assembly metrics. We also recommend basecalling in high accuracy mode. Greatest contig N50 is usually obtained with Shasta and Flye. Polishing/Correction is also recomendated (Racon and Medaka). https://nanoporetech.com/sites/default/files/s3/literature/human-genome-assembly-workflow.pdf


* Long reads simplify genome assembly, with the ability to span repeat-rich sequences (characteristic of  antimicrobial resistance genes) and structural variants. Nanopore sequencing also shows a lack of
bias in GC-rich regions, in contrast to other sequencing platforms. To perform microbial genome assembly, we suggest using the third-party de novo assembly tool Flye. We also recommend one round of polishing with Medaka. https://nanoporetech.com/sites/default/files/s3/literature/microbial-genome-assembly-workflow.pdf 

Flye  https://github.com/fenderglass/Flye 

Canu  https://canu.readthedocs.io/en/latest/quick-start.html

Miniasm  https://github.com/lh3/miniasm + Minipolish version https://github.com/rrwick/Minipolish

Shasta  https://github.com/chanzuckerberg/shasta

Smartdenovo  https://github.com/ruanjue/smartdenovo

Raven  https://github.com/lbcb-sci/raven

### 1.1 Assembly using Flye

We are going to assembly some Clones by using Flye https://github.com/fenderglass/Flye

Flye generates the concatenation of multiple disjoint genomic segments called disjointigs to build a repeat graph. Reads are mapped to this repeat graph to resolve conflicts (unbridged repeats) and output contigs.

In [5]:
# go to training folder
USER="jovyan"
cd /home/${USER}/SG-ONT-2021/

In [ ]:
mkdir -p /home/${USER}/SG-ONT-2021/
cd /home/${USER}/SG-ONT-2021/
time flye --nano-raw DATA/fastq_pass/Clone10.fastq --genome-size 1000000 --out-dir FLYE_Clone10 --threads 1

### 1.2 Assembly using Raven

In [11]:
# create a repertory for save raven assembly results
mkdir -p /home/${USER}/SG-ONT-2021/RAVEN_Clone10
cd /home/${USER}/SG-ONT-2021/RAVEN_Clone10/
pwd

/home/jovyan/SG-ONT-2021/RAVEN_Clone10


In [13]:
#launch raven
time raven -p 0 --graphical-fragment-assembly Clone10_raven.gfa -t 1 ../DATA/fastq_pass/Clone10.fastq > Clone10_raven.fasta

[raven::] loaded 11235 sequences 1.127538s
[raven::Graph::Construct] minimized 0 - 11235 / 11235 10.125334s
[raven::Graph::Construct] mapped sequences 53.893087s
[raven::Graph::Construct] annotated piles 0.389326s
[raven::Graph::Construct] removed contained sequences 0.019384s
[raven::Graph::Construct] removed chimeric sequences 0.011502s
[raven::Graph::Construct] reached checkpoint 0.117997s
[raven::Graph::Construct] minimized 0 - 214 / 214 1.254464s
[raven::Graph::Construct] mapped valid sequences 2.020366s
[raven::Graph::Construct] updated overlaps 0.000533s
[raven::Graph::Construct] removed false overlaps 0.027343s
[raven::Graph::Construct] stored 428 nodes 0.076212s
[raven::Graph::Construct] stored 3100 edges 0.003386s
[raven::Graph::Construct] reached checkpoint 0.141672s
[raven::Graph::Construct] 68.080606s
[raven::Graph::Assemble] removed transitive edges 0.003158s
[raven::Graph::Assemble] reached checkpoint 0.074192s
[raven::Graph::Assemble] removed tips and bubbles 0.047092s


* How many contigs were obtained by Flye and Raven ? Please fill in results in the shared file.
* Calculate first statistics about assemblies: N50 and lenght contig mean.

# 2. Polishing assemblies with Racon

Racon corrects raw contigs generated by rapid assembly methods with original ONT reads.

From 2 to 4 racon rounds are usually used by the community. 

Polish contigs assembled by Flye.

In [12]:
racon

[racon::] error: missing input file(s)!
usage: racon [options ...] <sequences> <overlaps> <target sequences>

    #default output is stdout
    <sequences>
        input file in FASTA/FASTQ format (can be compressed with gzip)
        containing sequences used for correction
    <overlaps>
        input file in MHAP/PAF/SAM format (can be compressed with gzip)
        containing overlaps between sequences and target sequences
    <target sequences>
        input file in FASTA/FASTQ format (can be compressed with gzip)
        containing sequences which will be corrected

    options:
        -u, --include-unpolished
            output unpolished target sequences
        -f, --fragment-correction
            perform fragment correction instead of contig polishing
            (overlaps file should contain dual/self overlaps!)
        -w, --window-length <int>
            default: 500
            size of window on which POA is performed
        -q, --quality-threshold <float>
            

: 1

# 3. Assemblies correction with Medaka

Correction can improve the consensus sequence for a draft genome assembly.

Medaka uses fast5 files to correct contigs using trained models. These models are free available.

Medaka allows you train a model from using fast5 of your favorite specie. You can use it directly to obtain a consensus from you favorite organism.

In [11]:
medaka

bash: medaka: command not found


: 127

### 1.3 Correct assemblies with Medaka